In [1]:
from typing import List, TypedDict
from langchain_core.documents import Document
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langgraph.graph import START, END, StateGraph
from langchain_naver import ChatClovaX, ClovaXEmbeddings
from langchain_milvus import Milvus

/opt/anaconda3/envs/gyu/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
llm = ChatClovaX(
    model="HCX-DASH-002", 
    max_tokens=1024,
    temperature=0,
    api_key=""
)

embeddings = ClovaXEmbeddings(
    model="bge-m3",
    api_key=""
)

In [ ]:
vectordb = Milvus(
    embedding_function=embeddings,
    connection_args={
        "uri": "https://in03-6919b557b41d797.serverless.gcp-us-west1.cloud.zilliz.com",   
        "token": ""          
    },
    collection_name="data_v2",                   
    auto_id=True
)

In [4]:
retriever = vectordb.as_retriever(
    search_type="similarity",
    search_kwargs={"k": 5}
)

In [5]:
class GraphState(TypedDict):
    question: str             
    context: List[Document]           

In [6]:
def retrieve_node(state: GraphState):
    question = state["question"]

    documents = retriever.invoke(question)
    
    return {"context": documents}

In [8]:
workflow = StateGraph(GraphState)

workflow.add_node("retrieve", retrieve_node)

workflow.add_edge(START, "retrieve")
workflow.add_edge("retrieve", END)

app = workflow.compile()

In [12]:
# generation은 빠져있습니다.